In [9]:
#Right now we could maybe widdle down to movies above a certain rating + number of ratings

In [31]:
import pandas as pd
import csv
import time
import numpy as np

import itertools
df_ratings = pd.read_csv('data/ratings.tsv',sep = '\t')
titles = pd.read_csv('data/titles.tsv',sep='\t')

C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
titles.head()
movies = titles[titles['titleType']=='movie']
movies = list(movies['tconst'].unique())
print(len(movies))

573966


In [36]:

#read in the ratings data
print(np.mean(df_ratings['numVotes']))
print(np.mean(df_ratings['averageRating']))
avgVotes = np.mean(df_ratings['numVotes'])
df_ratings.head()
qualifiedMovies = []
len(df_ratings)
boolean_series = df_ratings.tconst.isin(movies)
df_ratings = df_ratings[boolean_series]
for row in df_ratings.iterrows():
    if row[1]['averageRating'] >= 8.5 and row[1]['numVotes'] >= avgVotes:
        qualifiedMovies.append(row[1]['tconst'])
    

955.1516708090234
6.900688579030731


In [37]:
print(len(qualifiedMovies))

272


In [50]:

#[lambda x: x['tconst'] in qualifiedMovies]

In [38]:
count = 0
freqActors = dict()
for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
    chunk = chunk[chunk['category']=='actor']
    chunk = chunk.groupby('tconst')
    for name, group in chunk:
        if name not in qualifiedMovies:
            continue
        else:
            actors = group['nconst'].unique()
            for actor in actors:
                if actor in freqActors:
                    freqActors[actor] += 1
                else:
                    freqActors[actor] = 1
    

5199


In [41]:
freqActorsFinal = dict()
for actor in freqActors:
    if freqActors[actor] >= 2:
        freqActorsFinal[actor] = freqActors[actor]

In [66]:
freqActorsFinal = dict(sorted(freqActorsFinal.items(), key=lambda item: item[1],reverse=True))


In [43]:
freq_pairs = dict()
for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
    chunk = chunk[chunk['category']=='actor']
    chunk = chunk.groupby('tconst')
    for name, group in chunk:
        freqActorsGroup = [actor for actor in group['nconst'] if actor in freqActorsFinal]
        freqPairsGroup = list(itertools.combinations(freqActorsGroup,2))
        for pair in freqPairsGroup:
            if pair in freq_pairs:
                freq_pairs[pair] += 1
            else:
                freq_pairs[pair] = 1

In [84]:
freq_pairs = dict(sorted(freq_pairs.items(), key=lambda item: item[1],reverse=True))
#('nm0482320', 'nm0007123'): 32,
count = 0
freq_pairsFinal = dict()
for pair in freq_pairs:
    if (pair[1],pair[0]) not in freq_pairsFinal:
        freq_pairsFinal[pair] = freq_pairs[pair]
for pair in freq_pairsFinal:
    if (names[names['nconst']==pair[0]]['birthYear']  >= 1960):
        print('ok')

TypeError: '>=' not supported between instances of 'str' and 'int'

In [75]:
names = pd.read_csv('data/names.tsv',sep = '\t')
titles = pd.read_csv('data/titles.tsv',sep = '\t')

KeyboardInterrupt: 

In [62]:
names[names['nconst']=='nm0007123']['primaryName']
#names[names['nconst']=='nm0007123']
#titles[titles['tconst']== 'tt0118655']

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
7096,nm0007123,Mammootty,1951,\N,"actor,producer,music_department","tt1573478,tt0097851,tt0230597,tt0271648"


In [33]:
titles['titleType'].unique()


array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'episode'],
      dtype=object)